---
title: C++ Primer Plus: ch8 References Variables
tags: 小书匠,Books-C++PrimerPlus,reference,rvalue,lvalue
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# C++ Primer Plus: ch8 References Variables

## Temporary variables, Reference Arguments and const[#const-temporary]

### const reference allows mismatching types and rvalues <!-- {#const-temporary} -->

When using `const reference`, C++ can generate a temporary variable in two kinds of  situations
1. when the assigned value mismatch the correct type of the const reference
2. when the assigned value is an rvalue

Here is an example of the assigned value mismatch the correct type of the const reference

In [1]:
#include <iostream>
using namespace std;

int a = 1;
const double& x = a; // a is of type int
                     // while reference x is of type double
                     // A temporary value will be created to which x will refer
// a's and x's addresses are different
cout << &a << endl;
cout << &x << endl;

0x10ed852b0
0x10ed852a8


In [2]:
int b = 1;
const int& y = b; // b is of type int
                  // y is of type int
                  // No temporary value will be created here
// b's and y's addresses are the same
cout << &b << endl;
cout << &y << endl;

0x10ed8544c
0x10ed8544c


Here is an example of const reference to an rvalue.

In [3]:
const int& x = 1;

### non-const values DOESN'T allow mismatching types and rvalues

Here are examples

In [4]:
#include <iostream>
using namespace std;

int a = 1;
const double& x = a;    // valid. A temporary value will be created
// double &y = a;       // error

In [5]:
const int& x = 1; // valid. A const reference can be bind to an rvalue
// int& y = 1;    // error non-const reference cannot be bind to rvalue

### Why does non-const reference and const reference act differently?

In short, if the intent of a reference is merely to reference the bound variable instead of modifying the variable, a temporary variable will be created and bound to the const reference.

While if the intent of a reference is to modify the bound variable, the temporary variable will not be created and bound to the const reference, because it will thwart the purpose of modifying the bound variable. In this case, C++ will throw an error to prohibit creating temporary variables.

## Use reference as return value

### Why return a reference

The main reason to return a reference is to avoid unneccessary copies. 

Returning a value will cause several copies. Let see what happens when return a value.

In [8]:
double m = sqrt(16.0);

Assume `sqrt` returns a value, 
1. returned value 4.0 will be copied into a temporary variable. 
2. Then this temporary variable will be used to initialize m, here may invoke at least one copy.

While if return a reference, not copies is needed. Thus, return a reference will boost the efficiency.

Note:
1. Accoarding to previous discussion, **functions returning a value will return an rvalue**.
2. In C++11, the return procedure described previously will be optimized with move sematic.

### Being careful about what a return reference refers to

The single most important point to remember when returning a reference is to avoid returning a reference to a memory location that ceases to exist when the function terminates.

In [6]:
int copy1(const int& a)
{
    int b = a;
    return b;
}

In [7]:
int& copy2(const int& a)
{
    int b = a;
    return b;
} // warning reference to stack memory associated with local variable 'b' returned

int x = 1;
int b1 = copy1(x);
int& b2 = copy2(x);
cout << b1 << endl;
cout << b2 << endl;

input_line_15:4:12: warning: reference to stack memory associated with local variable 'b' returned [-Wreturn-stack-address]
    return b;
           ^


1
1


Avoiding such problems are easy: do not return a reference to a memory location that ceases to exists when the function terminates.
There are two guidelines:
1. Returning references to the called parameters
2. Returning references to the allocated memory using new in the function.

## Use const when you can

There are mainly three reasons to use const if possible

1. Using a const reference protects the variable from being modified unexpectedly.

2. Using a const reference allows a function to process both const and non-const data.

3. Using a const reference allows the function to generate and use a temporary variable appropriately

### Passing a C-style string argument to a string object reference parameter

The following two examples illustrates passing a c-style string argument to a string object reference parameter.

In [13]:
void print1(const string& s)
{
   cout << s << endl; 
}

print1("hello world");  // Type mismatches. const string is expected while actual type is char*
                        // A temporary string object initialized with "hello world" will be created and referred to

hello world


In [15]:
void print2(string& s)
{
   cout << s << endl; 
} // error

print2("hello world"); // A non-const reference cannot will raise an error when type mismatch
                       // error: no matching function for call to 'print2'

input_line_23:6:2: error: no matching function for call to 'print2'
 print2("hello world"); // A non-const reference cannot will raise an error when type mismatch
 ^~~~~~
input_line_23:1:6: note: candidate function not viable: no known conversion from 'const char [12]' to 'std::__1::string &'
      (aka 'basic_string<char, char_traits<char>, allocator<char> > &') for 1st argument
void print2(string& s)
     ^


Interpreter Error: 

The reason is [here](#const-temporary). Non-const will prohibit creating temporary variable. Thus, prototype `void print2(string&)` will not match in the of type char*.

# References